In [1]:
library(grid)
library(rpart)
library(rpart.plot)
library(partykit)
library(lattice)
library(ggplot2)
library(caret)
library(dplyr)


Attaching package: ‘dplyr’

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



In [2]:
set.seed(1)

In [3]:
jRoadType <- c("都市間高速", "都市高速", "有料道路", "国道", "県道", "主要地方道", "一般道1、一般道2、一般道3", "その他")
jHighSpeeds <- c("都市間高速", "都市高速", "有料道路")

In [4]:
printf <- function(...) cat(sprintf(...))

In [5]:
# valid columns when high speed
invalids <- c('Time', 'Longitude', 'Latitude', 'Brake', 'Accel', 'RoadType', 'flag', 'TimeToCollision', "RoadFactor")
valuables <- c("RoadType","CurveAverage","Speed", "Curve100", "Curve150","MaxSpeed","RiskFactor","Curve","DistSignal","Pitch","AheadDistance","AverageVelocity","TimeToCollision","AccelerationSpeed", "Engine", "SteeringAngle", "TimeHeadway", "Jerk", "LaneCount", "DiffAvgSpeed", "EmptinessOfRoad", "RoadFactor")
isValidColumn <- function(x) ! x %in% invalids

In [6]:
trainAndPredict <- function(dfx, verbose=FALSE) {        
    train <- dfx[, c(valuables, "flag")]
    test <- dfx
    fit <- rpart(flag ~ ., data=train)
    p <- predict(fit, newdata=test, cp=0.018)
    predictedFlags <- colnames(p)[max.col(p, ties.method = "first")]

    showMatrix(test, predictedFlags)

    nerr <- sum(predictedFlags != test$flag)
}

In [7]:
showMatrix <- function(test, predictedFlags) {
    predictedRed <- test[predictedFlags == "Red", ]
    predictedBlue <- test[predictedFlags == "Blue", ]
    printf("Predict Red\n")
    print(predictedRed %>% group_by(flag, AccelOrBrake) %>% summarize(count=n()))
    printf("Predict Blue\n")
    print(predictedBlue %>% group_by(flag, AccelOrBrake) %>% summarize(count=n()))
}

In [8]:
createFormula <- function(invalids, allFeatures) {
    isValidColumn <- function(x) ! x %in% invalids
    
    features <- Filter(isValidColumn, allFeatures)
    return(paste("flag ~ ", paste(features, collapse=" + ")))
}

# Predict Reds

In [9]:
df <- read.csv("../data/middle/sp5.csv", stringsAsFactors=FALSE)

In [10]:
df$AccelOrBrake[df$flag == "RedA"] <- "Accel"
df$AccelOrBrake[df$flag == "RedB"] <- "Brake"
df$AccelOrBrake[df$flag == "BlueA"] <- "Accel"
df$AccelOrBrake[df$flag == "BlueB"] <- "Brake"

In [11]:
df$flag[df$flag == "RedA"] <- "Red"
df$flag[df$flag == "RedB"] <- "Red"
df$flag[df$flag == "BlueA"] <- "Blue"
df$flag[df$flag == "BlueB"] <- "Blue"
df$flag <- as.factor(df$flag)

In [12]:
allFeatures <- c(colnames(df))

In [13]:
roadTypes <- unique(df$RoadType)

In [15]:
for (i in c(0, 2, 3, 4, 5, 6, 7)) {
    printf("RoadType: %d (%s)\n", i, jRoadType[i+1])
    dfx <- df[df$RoadType == i, ]
    result <- trainAndPredict(dfx, verbose=TRUE)
}

RoadType: 0 (都市間高速)
Predict Red
Source: local data frame [4 x 3]
Groups: flag [?]

    flag AccelOrBrake count
  <fctr>        <chr> <int>
1   Blue        Accel     5
2   Blue        Brake     4
3    Red        Accel    44
4    Red        Brake     4
Predict Blue
Source: local data frame [4 x 3]
Groups: flag [?]

    flag AccelOrBrake count
  <fctr>        <chr> <int>
1   Blue        Accel     8
2   Blue        Brake     9
3    Red        Accel     6
4    Red        Brake     1
RoadType: 2 (有料道路)
Predict Red
Source: local data frame [4 x 3]
Groups: flag [?]

    flag AccelOrBrake count
  <fctr>        <chr> <int>
1   Blue        Accel     2
2   Blue        Brake     4
3    Red        Accel    21
4    Red        Brake     9
Predict Blue
Source: local data frame [2 x 3]
Groups: flag [?]

    flag AccelOrBrake count
  <fctr>        <chr> <int>
1   Blue        Brake    10
2    Red        Brake     1
RoadType: 3 (国道)
Predict Red
Source: local data frame [4 x 3]
Groups: flag [?]

    flag Ac